In [1]:
# Install required packages
!pip install livekit-agents exa-py cerebras-cloud-sdk langchain langgraph

# Set environment variables
# export CEREBRAS_API_KEY="your_cerebras_key"
# export LIVEKIT_API_KEY="your_livekit_key" 
# export CARTESIA_API_KEY="your_cartesia_key"
# export EXA_API_KEY="your_exa_key"


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


1. Voice Sales Agent with LiveKit and Cartesia

In [2]:
#solution 1

In [3]:
import os
import asyncio
from typing import Dict, List, Optional
from dataclasses import dataclass
from livekit.agents import (
    Agent, 
    LLM, 
    STT, 
    TTS, 
    VoiceActivityDetector,
    JobContext,
    llm
)
from livekit.agents.pipeline import VoicePipeline

# Step 1: Configuration and API Keys
@dataclass
class AgentConfig:
    cerebras_api_key: str
    livekit_api_key: str
    cartesia_api_key: str
    llm_model: str = "llama-3-70b"
    stt_model: str = "cartesia-whisper-v3"
    tts_model: str = "cartesia-sonic"

class SalesContext:
    def __init__(self):
        self.product_info = {}
        self.pricing_data = {}
        self.objection_handlers = {}
        self.benefits = []
    
    def load_context(self, context_data: Dict):
        """Load sales context for the agent"""
        self.product_info = context_data.get("product_description", {})
        self.pricing_data = context_data.get("pricing", {})
        self.objection_handlers = context_data.get("objection_handlers", {})
        self.benefits = context_data.get("key_benefits", [])
    
    def get_context_prompt(self) -> str:
        """Create context prompt for the LLM"""
        context_parts = []
        
        if self.product_info:
            context_parts.append(f"Product: {self.product_info.get('name', '')}")
            context_parts.append(f"Description: {self.product_info.get('description', '')}")
        
        if self.benefits:
            context_parts.append("Key Benefits:")
            for benefit in self.benefits:
                context_parts.append(f"- {benefit}")
        
        if self.pricing_data:
            context_parts.append("Pricing:")
            for tier, price in self.pricing_data.items():
                context_parts.append(f"- {tier}: {price}")
        
        if self.objection_handlers:
            context_parts.append("Common Objection Responses:")
            for objection, response in self.objection_handlers.items():
                context_parts.append(f"Objection: '{objection}'")
                context_parts.append(f"Response: '{response}'")
        
        return "\n".join(context_parts)

class SalesAgent(Agent):
    def __init__(self, config: AgentConfig, context: SalesContext):
        self.config = config
        self.context = context
        
        # Initialize AI components
        self.llm = self._setup_llm()
        self.stt = self._setup_stt()
        self.tts = self._setup_tts()
        self.vad = self._setup_vad()
        
        # Sales-specific instructions
        self.instructions = self._create_instructions()
        
        super().__init__(
            llm=self.llm,
            stt=self.stt,
            tts=self.tts,
            vad=self.vad,
            instructions=self.instructions
        )
    
    def _setup_llm(self) -> LLM:
        """Configure Cerebras LLM"""
        return LLM.create(
            provider="cerebras",
            model=self.config.llm_model,
            api_key=self.config.cerebras_api_key
        )
    
    def _setup_stt(self) -> STT:
        """Configure Cartesia Speech-to-Text"""
        return STT.create(
            provider="cartesia",
            model=self.config.stt_model,
            api_key=self.config.cartesia_api_key
        )
    
    def _setup_tts(self) -> TTS:
        """Configure Cartesia Text-to-Speech"""
        return TTS.create(
            provider="cartesia",
            model=self.config.tts_model,
            api_key=self.config.cartesia_api_key,
            voice_id="friendly-sales"  # Pre-configured sales voice
        )
    
    def _setup_vad(self) -> VoiceActivityDetector:
        """Configure Voice Activity Detection"""
        return VoiceActivityDetector.create(provider="cilero")
    
    def _create_instructions(self) -> str:
        """Create comprehensive sales agent instructions"""
        base_instructions = f"""
        You are a professional sales agent communicating by voice with potential customers.
        
        IMPORTANT RULES:
        1. Speak naturally and conversationally - no bullet points or lists
        2. Be helpful, engaging, and professional
        3. Keep responses concise but informative
        4. Only use information from the provided context
        5. If asked about something not in context, politely say you don't have that information
        
        CONTEXT INFORMATION:
        {self.context.get_context_prompt()}
        
        SALES GUIDELINES:
        - Build rapport with customers
        - Understand their needs before pushing products
        - Handle objections professionally using the provided responses
        - Focus on benefits and value, not just features
        - Guide the conversation toward a positive outcome
        
        CONVERSATION FLOW:
        1. Greet warmly and introduce yourself
        2. Ask about their needs or challenges
        3. Listen actively and ask follow-up questions
        4. Present relevant solutions
        5. Address concerns professionally
        6. Guide toward next steps
        """
        return base_instructions
    
    async def on_enter(self, ctx: JobContext):
        """Triggered when someone joins the conversation"""
        greeting = """
        Hello! Welcome to our sales consultation. I'm your AI sales assistant. 
        I'm here to help you find the perfect solution for your needs. 
        What brings you here today, and how can I assist you?
        """
        await ctx.reply(greeting)

# Multi-Agent System for Specialized Sales
class TechnicalSpecialistAgent(SalesAgent):
    def _create_instructions(self) -> str:
        base_instructions = super()._create_instructions()
        specialized_instructions = """
        SPECIALIZATION: Technical Expert
        - Focus on technical specifications and integration details
        - Explain complex concepts in simple terms
        - Provide technical comparisons and implementation guidance
        - Address technical concerns and compatibility issues
        """
        return base_instructions + specialized_instructions

class PricingSpecialistAgent(SalesAgent):
    def _create_instructions(self) -> str:
        base_instructions = super()._create_instructions()
        specialized_instructions = """
        SPECIALIZATION: Pricing and ROI Expert
        - Focus on cost-benefit analysis and ROI calculations
        - Explain pricing tiers and value propositions
        - Handle budget discussions and payment options
        - Provide case studies and success metrics
        """
        return base_instructions + specialized_instructions

# Main Application Entry Point
class SalesAgentApplication:
    def __init__(self, config: AgentConfig):
        self.config = config
        self.agents = {}
        self.setup_agents()
    
    def setup_agents(self):
        """Initialize all specialized agents"""
        # Load common sales context
        context = SalesContext()
        context.load_context(self._load_sales_data())
        
        # Create specialized agents
        self.agents["greeting"] = SalesAgent(config, context)
        self.agents["technical"] = TechnicalSpecialistAgent(config, context)
        self.agents["pricing"] = PricingSpecialistAgent(config, context)
    
    def _load_sales_data(self) -> Dict:
        """Load product and sales data"""
        return {
            "product_description": {
                "name": "AI Sales Platform",
                "description": "Cutting-edge AI platform for automated sales and customer engagement"
            },
            "key_benefits": [
                "Increase conversion rates by 40%",
                "Reduce sales cycle time by 60%",
                "24/7 customer engagement",
                "Personalized customer interactions"
            ],
            "pricing": {
                "Starter": "$99/month",
                "Professional": "$299/month", 
                "Enterprise": "Custom pricing"
            },
            "objection_handlers": {
                "It's too expensive": "I understand cost is important. Let's look at the ROI - most clients see 3x return in the first six months.",
                "I need to think about it": "That's completely understandable. What specific concerns can I address to help with your decision?",
                "I'm not sure we need this": "Many of our clients felt the same initially. Would you like me to share some case studies showing similar companies' results?"
            }
        }
    
    async def start_agent(self, agent_type: str = "greeting"):
        """Start the specified agent"""
        agent = self.agents.get(agent_type, self.agents["greeting"])
        
        # Connect to LiveKit room
        pipeline = VoicePipeline(agent)
        await pipeline.connect()
        
        return pipeline

# Usage Example
async def main():
    # Configuration
    config = AgentConfig(
        cerebras_api_key=os.getenv("CEREBRAS_API_KEY"),
        livekit_api_key=os.getenv("LIVEKIT_API_KEY"),
        cartesia_api_key=os.getenv("CARTESIA_API_KEY")
    )
    
    # Create application
    app = SalesAgentApplication(config)
    
    # Start the sales agent
    pipeline = await app.start_agent("greeting")
    
    print("Sales agent is running...")
    # The agent will now handle conversations automatically

if __name__ == "__main__":
    asyncio.run(main())

ImportError: cannot import name 'LLM' from 'livekit.agents' (/home/daniel/anaconda3/lib/python3.12/site-packages/livekit/agents/__init__.py)

2. Deep Research Agent with Exa

In [ ]:
#soluiton 2

In [ ]:
import os
import asyncio
import json
from typing import Dict, List, Optional, Any
from dataclasses import dataclass
from exa_py import Exa
from cerebras.cloud.sdk import Cerebras

@dataclass
class ResearchConfig:
    cerebras_api_key: str
    exa_api_key: str
    model: str = "llama-3-70b"
    max_sources: int = 5
    content_length: int = 2000

class ResearchAgent:
    def __init__(self, config: ResearchConfig):
        self.config = config
        self.exa = Exa(api_key=config.exa_api_key)
        self.cerebras = Cerebras(api_key=config.cerebras_api_key)
        
    async def search_web(self, query: str, num_results: int = 5) -> List[Dict]:
        """Search the web using Exa API"""
        try:
            response = self.exa.search_and_contents(
                query,
                type="auto",  # Auto-choose between keyword and neural search
                text=True,    # Get full content
                num_results=num_results
            )
            
            results = []
            for result in response.results:
                if len(result.text or '') > 200:  # Filter substantial content
                    results.append({
                        'title': result.title,
                        'url': result.url,
                        'content': result.text,
                        'published_date': result.published_date
                    })
            
            return results
        except Exception as e:
            print(f"Search error: {e}")
            return []
    
    async def ask_ai(self, question: str, context: str, temperature: float = 0.2) -> str:
        """Query Cerebras LLM with context"""
        prompt = f"""
        RESEARCH TASK: Analyze the following sources and answer the question.
        
        QUESTION: {question}
        
        SOURCES:
        {context}
        
        Please provide:
        1. A comprehensive summary of key findings
        2. Specific insights and patterns identified
        3. Any contradictions or uncertainties found
        4. Practical implications or recommendations
        
        Format your response clearly with sections.
        """
        
        try:
            response = self.cerebras.completions.create(
                model=self.config.model,
                prompt=prompt,
                max_tokens=2000,
                temperature=temperature,
                stop=["###", "END"]
            )
            
            return response.choices[0].text.strip()
        except Exception as e:
            return f"Error generating response: {e}"
    
    async def research_topic(self, topic: str) -> Dict[str, Any]:
        """Basic research function - single search and analysis"""
        print(f"Researching: {topic}")
        
        # Step 1: Search for sources
        sources = await self.search_web(topic, self.config.max_sources)
        
        if not sources:
            return {"error": "No substantial sources found"}
        
        # Step 2: Prepare context for LLM
        context = self._prepare_context(sources)
        
        # Step 3: Analyze with LLM
        analysis = await self.ask_ai(topic, context)
        
        return {
            "topic": topic,
            "sources_used": len(sources),
            "analysis": analysis,
            "source_urls": [s['url'] for s in sources]
        }
    
    async def deep_research_topic(self, topic: str, max_iterations: int = 2) -> Dict[str, Any]:
        """Advanced research with recursive gap analysis"""
        print(f"Deep researching: {topic}")
        
        all_sources = []
        current_question = topic
        
        for iteration in range(max_iterations):
            print(f"Iteration {iteration + 1}")
            
            # Search for current question
            new_sources = await self.search_web(current_question, 3)
            all_sources.extend(new_sources)
            
            if not new_sources:
                break
                
            # Analyze current state
            context = self._prepare_context(all_sources)
            initial_analysis = await self.ask_ai(current_question, context)
            
            # Identify knowledge gaps
            gap_analysis = await self.identify_gaps(topic, initial_analysis, context)
            
            if not gap_analysis.get('needs_followup', False):
                break
                
            current_question = gap_analysis['followup_question']
        
        # Final comprehensive analysis
        final_context = self._prepare_context(all_sources)
        final_analysis = await self.ask_ai(topic, final_context)
        
        return {
            "topic": topic,
            "iterations": iteration + 1,
            "total_sources": len(all_sources),
            "final_analysis": final_analysis,
            "source_urls": [s['url'] for s in all_sources]
        }
    
    async def identify_gaps(self, original_question: str, current_analysis: str, context: str) -> Dict:
        """Identify gaps in current research and suggest follow-up questions"""
        gap_prompt = f"""
        Based on the current research analysis, identify the most important knowledge gap.
        
        ORIGINAL QUESTION: {original_question}
        
        CURRENT ANALYSIS:
        {current_analysis}
        
        SOURCES ANALYZED:
        {context}
        
        Please provide:
        1. Assessment of whether follow-up research is needed
        2. The most important unanswered question
        3. Why this gap is significant
        
        Respond in JSON format:
        {{
            "needs_followup": true/false,
            "followup_question": "specific question to research",
            "gap_significance": "why this gap matters"
        }}
        """
        
        try:
            response = self.cerebras.completions.create(
                model=self.config.model,
                prompt=gap_prompt,
                max_tokens=500,
                temperature=0.1
            )
            
            result_text = response.choices[0].text.strip()
            return json.loads(result_text)
        except:
            return {"needs_followup": False, "followup_question": "", "gap_significance": ""}
    
    def _prepare_context(self, sources: List[Dict]) -> str:
        """Prepare source content for LLM context"""
        context_parts = []
        for i, source in enumerate(sources, 1):
            context_parts.append(f"SOURCE {i}: {source['title']}")
            context_parts.append(f"URL: {source['url']}")
            context_parts.append(f"CONTENT: {source['content'][:1000]}...")  # Truncate long content
            context_parts.append("---")
        
        return "\n".join(context_parts)

# Multi-Agent Research System (Anthrophic-style)
class MultiAgentResearchSystem:
    def __init__(self, config: ResearchConfig):
        self.config = config
        self.agents = {
            'lead': ResearchAgent(config),
            'technical': ResearchAgent(config),
            'market': ResearchAgent(config),
            'synthesis': ResearchAgent(config)
        }
    
    async def research_with_agents(self, topic: str) -> Dict:
        """Conduct research using multiple specialized agents"""
        print("Starting multi-agent research...")
        
        # Lead agent breaks down the topic
        breakdown = await self._breakdown_topic(topic)
        
        # Run specialized research in parallel
        tasks = []
        for subtopic in breakdown.get('subtopics', []):
            if 'technical' in subtopic.lower():
                tasks.append(self.agents['technical'].deep_research_topic(subtopic))
            elif 'market' in subtopic.lower() or 'business' in subtopic.lower():
                tasks.append(self.agents['market'].deep_research_topic(subtopic))
            else:
                tasks.append(self.agents['lead'].deep_research_topic(subtopic))
        
        # Wait for all research to complete
        results = await asyncio.gather(*tasks, return_exceptions=True)
        
        # Synthesize final report
        synthesis = await self._synthesize_results(topic, results)
        
        return synthesis
    
    async def _breakdown_topic(self, topic: str) -> Dict:
        """Break down complex topic into subtopics"""
        breakdown_prompt = f"""
        Break down this research topic into specialized subtopics for parallel research:
        
        TOPIC: {topic}
        
        Provide a JSON response with:
        - main_topic: the original topic
        - subtopics: list of 3-5 specialized areas to research
        - research_focus: brief description of what each subtopic should cover
        """
        
        # Implementation similar to ask_ai method
        # ... (code for LLM interaction)
        return {"subtopics": [topic]}  # Simplified
    
    async def _synthesize_results(self, topic: str, results: List) -> Dict:
        """Synthesize results from multiple research agents"""
        synthesis_prompt = f"""
        Synthesize research findings from multiple specialized agents:
        
        ORIGINAL TOPIC: {topic}
        
        RESEARCH RESULTS: {json.dumps(results, indent=2)}
        
        Create a comprehensive final report that:
        1. Integrates findings from all sources
        2. Identifies key patterns and insights
        3. Highlights areas of consensus and disagreement
        4. Provides actionable recommendations
        """
        
        # Implementation similar to ask_ai method
        return {"synthesis": "Integrated research report"}

# Usage Example
async def research_demo():
    config = ResearchConfig(
        cerebras_api_key=os.getenv("CEREBRAS_API_KEY"),
        exa_api_key=os.getenv("EXA_API_KEY")
    )
    
    agent = ResearchAgent(config)
    
    # Basic research
    result = await agent.research_topic("AI in healthcare 2024")
    print("Basic Research Result:")
    print(json.dumps(result, indent=2))
    
    # Deep research
    deep_result = await agent.deep_research_topic("quantum computing advances 2024")
    print("\nDeep Research Result:")
    print(json.dumps(deep_result, indent=2))

if __name__ == "__main__":
    asyncio.run(research_demo())

3. User Research Agent with LangChain

In [ ]:
#solution 3

In [ ]:
import os
import asyncio
from typing import Dict, List, Optional, Any
from dataclasses import dataclass
from langchain.schema import BaseMessage, HumanMessage, AIMessage
from langchain.chat_models import ChatOpenAI
from langgraph.graph import StateGraph, END
from pydantic import BaseModel

@dataclass
class UserResearchConfig:
    cerebras_api_key: str
    model: str = "llama-3-70b"
    num_personas: int = 5
    questions_per_interview: int = 3

# State management using Pydantic models
class InterviewState(BaseModel):
    research_question: str
    personas: List[Dict[str, Any]] = []
    interviews: Dict[str, List[Dict]] = {}
    current_persona_index: int = 0
    current_question_index: int = 0
    insights: Dict[str, Any] = {}
    follow_up_context: Dict[str, Any] = {}

class UserResearchAgent:
    def __init__(self, config: UserResearchConfig):
        self.config = config
        self.llm = self._setup_llm()
        self.workflow = self._build_workflow()
    
    def _setup_llm(self):
        """Configure Cerebras LLM through LangChain"""
        # Note: This would use a custom Cerebras LangChain integration
        # For now, using OpenAI format as placeholder
        return ChatOpenAI(
            model_name=self.config.model,
            openai_api_base="https://api.cerebras.ai/v1",
            openai_api_key=self.config.cerebras_api_key
        )
    
    def _build_workflow(self) -> StateGraph:
        """Build LangGraph workflow for user research"""
        workflow = StateGraph(InterviewState)
        
        # Add nodes
        workflow.add_node("configure_research", self.configure_research)
        workflow.add_node("generate_personas", self.generate_personas)
        workflow.add_node("conduct_interview", self.conduct_interview)
        workflow.add_node("synthesize_insights", self.synthesize_insights)
        workflow.add_node("generate_followup", self.generate_followup)
        
        # Define workflow flow
        workflow.set_entry_point("configure_research")
        workflow.add_edge("configure_research", "generate_personas")
        workflow.add_edge("generate_personas", "conduct_interview")
        
        # Conditional edges for interview flow
        workflow.add_conditional_edges(
            "conduct_interview",
            self.should_continue_interview,
            {
                "next_persona": "conduct_interview",
                "next_question": "generate_followup", 
                "synthesize": "synthesize_insights"
            }
        )
        
        workflow.add_edge("generate_followup", "conduct_interview")
        workflow.add_edge("synthesize_insights", END)
        
        return workflow.compile()
    
    async def configure_research(self, state: InterviewState) -> InterviewState:
        """Node 1: Configure research parameters"""
        print("Configuring research...")
        
        # Generate interview questions based on research topic
        questions_prompt = f"""
        Generate {self.config.questions_per_interview} open-ended interview questions 
        for user research on: {state.research_question}
        
        Questions should:
        - Reveal user motivations and pain points
        - Be open-ended to encourage detailed responses
        - Cover different aspects of the topic
        - Avoid yes/no questions
        
        Return as JSON list of questions.
        """
        
        response = await self.llm.agenerate([questions_prompt])
        questions = self._parse_json_response(response)
        
        state.interviews["questions"] = questions
        return state
    
    async def generate_personas(self, state: InterviewState) -> InterviewState:
        """Node 2: Generate diverse user personas"""
        print("Generating personas...")
        
        persona_prompt = f"""
        Generate {self.config.num_personas} diverse user personas for research on: {state.research_question}
        
        Each persona should include:
        - Name and age
        - Background and profession
        - Technical proficiency level
        - Key motivations and pain points
        - Communication style
        - Specific needs related to the research topic
        
        Make personas diverse in age, background, and perspectives.
        Return as JSON list of personas.
        """
        
        response = await self.llm.agenerate([persona_prompt])
        state.personas = self._parse_json_response(response)
        
        # Initialize interview tracking
        for persona in state.personas:
            state.interviews[persona['name']] = []
        
        return state
    
    async def conduct_interview(self, state: InterviewState) -> InterviewState:
        """Node 3: Conduct AI-powered interview with persona"""
        current_persona = state.personas[state.current_persona_index]
        current_question = state.interviews["questions"][state.current_question_index]
        
        print(f"Interviewing {current_persona['name']}...")
        
        # Prepare interview context
        interview_prompt = self._create_interview_prompt(
            current_persona, 
            current_question,
            state
        )
        
        # Get persona response
        response = await self.llm.agenerate([interview_prompt])
        persona_response = response.generations[0][0].text
        
        # Store interview data
        interview_data = {
            "question": current_question,
            "response": persona_response,
            "question_index": state.current_question_index,
            "follow_up_context": state.follow_up_context.get(current_persona['name'], {})
        }
        
        state.interviews[current_persona['name']].append(interview_data)
        
        return state
    
    async def generate_followup(self, state: InterviewState) -> InterviewState:
        """Node 4: Generate contextual follow-up questions"""
        current_persona = state.personas[state.current_persona_index]
        recent_responses = state.interviews[current_persona['name']][-2:]  # Last 2 responses
        
        if len(recent_responses) < 2:
            # Move to next question if not enough context for follow-up
            state.current_question_index += 1
            return state
        
        followup_prompt = f"""
        Based on the persona's recent responses, generate a natural follow-up question.
        
        PERSONA: {current_persona}
        
        RECENT RESPONSES: {recent_responses}
        
        Create a follow-up question that:
        - Digs deeper into interesting points
        - Explores motivations behind statements
        - Maintains natural conversation flow
        - Is open-ended and exploratory
        
        Return only the follow-up question.
        """
        
        response = await self.llm.agenerate([followup_prompt])
        followup_question = response.generations[0][0].text.strip()
        
        # Add follow-up to questions list
        state.interviews["questions"].append(followup_question)
        state.current_question_index = len(state.interviews["questions"]) - 1
        
        # Update follow-up context
        persona_name = current_persona['name']
        if persona_name not in state.follow_up_context:
            state.follow_up_context[persona_name] = {}
        state.follow_up_context[persona_name]["last_followup"] = followup_question
        
        return state
    
    async def synthesize_insights(self, state: InterviewState) -> InterviewState:
        """Node 5: Synthesize insights from all interviews"""
        print("Synthesizing insights...")
        
        synthesis_prompt = self._create_synthesis_prompt(state)
        response = await self.llm.agenerate([synthesis_prompt])
        
        insights = self._parse_json_response(response)
        state.insights = insights
        
        print("Research complete! Insights generated.")
        return state
    
    def should_continue_interview(self, state: InterviewState) -> str:
        """Determine next step in interview flow"""
        # Check if all personas have been interviewed
        if state.current_persona_index < len(state.personas) - 1:
            state.current_persona_index += 1
            state.current_question_index = 0
            return "next_persona"
        
        # Check if all questions have been asked
        if state.current_question_index < len(state.interviews["questions"]) - 1:
            state.current_question_index += 1
            state.current_persona_index = 0  # Start with first persona for next question
            return "next_question"
        
        # All interviews complete
        return "synthesize"
    
    def _create_interview_prompt(self, persona: Dict, question: str, state: InterviewState) -> str:
        """Create prompt for persona response generation"""
        return f"""
        You are role-playing as: {persona}
        
        Research Context: {state.research_question}
        
        Interview Question: {question}
        
        Respond naturally as this persona would:
        - Stay true to the persona's background and characteristics
        - Provide detailed, thoughtful responses
        - Express personal opinions and experiences
        - Be authentic and believable
        
        Previous responses in this interview: {state.interviews.get(persona['name'], [])}
        
        Response:
        """
    
    def _create_synthesis_prompt(self, state: InterviewState) -> str:
        """Create prompt for insight synthesis"""
        all_interviews = []
        for persona_name, interviews in state.interviews.items():
            if persona_name != "questions":  # Skip questions key
                all_interviews.append({
                    "persona": persona_name,
                    "responses": interviews
                })
        
        return f"""
        Analyze the following user research interviews and synthesize key insights.
        
        RESEARCH TOPIC: {state.research_question}
        
        INTERVIEW DATA: {all_interviews}
        
        Provide a comprehensive analysis including:
        1. Key patterns and themes across personas
        2. Surprising or unexpected findings
        3. User pain points and frustrations
        4. Opportunities and recommendations
        5. Demographic variations in responses
        
        Format as JSON with clear sections.
        """
    
    def _parse_json_response(self, response) -> Any:
        """Parse JSON response from LLM"""
        # Implementation for extracting JSON from LLM response
        try:
            import json
            text = response.generations[0][0].text
            # Extract JSON from text if needed
            start = text.find('{')
            end = text.rfind('}') + 1
            if start != -1 and end != 0:
                return json.loads(text[start:end])
        except:
            pass
        return []

# Enhanced Research System with Multi-Question Interviews
class EnhancedUserResearchAgent(UserResearchAgent):
    def __init__(self, config: UserResearchConfig):
        super().__init__(config)
        # Additional enhancements for multi-question flow
    
    async def conduct_interview(self, state: InterviewState) -> InterviewState:
        """Enhanced interview with conversation memory"""
        # Implementation with conversation memory and context tracking
        return await super().conduct_interview(state)

# Usage Example
async def user_research_demo():
    config = UserResearchConfig(
        cerebras_api_key=os.getenv("CEREBRAS_API_KEY"),
        num_personas=3,
        questions_per_interview=2
    )
    
    agent = UserResearchAgent(config)
    
    # Initialize research state
    initial_state = InterviewState(
        research_question="How can we improve mobile banking apps for young professionals?"
    )
    
    # Run the research workflow
    final_state = await agent.workflow.ainvoke(initial_state)
    
    print("Research Insights:")
    print(final_state.insights)

if __name__ == "__main__":
    asyncio.run(user_research_demo())